In [14]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)

#note: for graphviz, need to install it separately, and also install the pygraphviz package.
#also, for plotly need to sign up and set the api key. refer to plotly set up guide on their website

Task 3 Viz 1: Showing a line chart animation of the rankings of sons in a particular region.

Try 1: Using Plotly line chart animations

In [2]:
rankings_file_path = "../../data/Daily_Ranking.csv"

In [3]:
df = pd.read_csv(rankings_file_path)

In [4]:
#first create dictionary of region code to country name for simplicity later
all_region_codes = ['ec', 'fr', 'ar', 'fi', 'no', 'it', 'lt', 'ph', 'tw', 'nz', 'ee', 'tr', 'us', 'sv', 'cr', 'de', 'cl', 'jp', 'br', 'hn', 'gt', 'ch', 'hu', 'ca', 'pe', 'be', 'my', 'dk', 'bo', 'pl', 'at', 'pt', 'se', 'mx', 'pa', 'uy', 'is', 'es', 'cz', 'ie', 'nl', 'sk', 'co', 'sg', 'id', 'do', 'lu', 'gb', 'py', 'au', 'lv', 'gr', 'hk']
df_countries = pd.read_json("../../data/countries.json")
df_countries = df_countries.transpose()
country_names = {r:df_countries.loc[r.upper()]['name'] for r in all_region_codes}

In [5]:
print("Select Region to Show in the Animation")
region_selector = widgets.Dropdown(
    options=[country_names[i] for i in all_region_codes],
    value='United States',
    description='Region:')
display(region_selector)

Select Region to Show in the Animation


Dropdown(description='Region:', index=12, options=('Ecuador', 'France', 'Argentina', 'Finland', 'Norway', 'Italy', 'Lithuania', 'Philippines', 'Taiwan', 'New Zealand', 'Estonia', 'Turkey', 'United States', 'El Salvador', 'Costa Rica', 'Germany', 'Chile', 'Japan', 'Brazil', 'Honduras', 'Guatemala', 'Switzerland', 'Hungary', 'Canada', 'Peru', 'Belgium', 'Malaysia', 'Denmark', 'Bolivia', 'Poland', 'Austria', 'Portugal', 'Sweden', 'Mexico', 'Panama', 'Uruguay', 'Iceland', 'Spain', 'Czech Republic', 'Ireland', 'Netherlands', 'Slovakia', 'Colombia', 'Singapore', 'Indonesia', 'Dominican Republic', 'Luxembourg', 'United Kingdom', 'Paraguay', 'Australia', 'Latvia', 'Greece', 'Hong Kong'), value='United States')

In [6]:
all_region_codes[region_selector.index]
df.columns

Index(['Position', 'Track Name', 'Artist', 'Streams', 'URL', 'Date', 'Region'], dtype='object')

In [ ]:
#run this cell to show the animation!
region_chosen = all_region_codes[region_selector.index]
df_specific_region = df[df['Region'] == region_chosen]
df_specific_region = df_specific_region[df_specific_region["Position"] <= 10]   #only consider the top 10 positions
region_days = sorted(df_specific_region["Date"].unique().tolist())
if len(region_days) > 100:
    print("truncating no. of days to just 100 for better performance")
    region_days = region_days[:100]

figure = {
    'data': [],
    'layout': {},
    'frames': []
}

# fill in most of layout
figure['layout']['hovermode'] = 'closest'
figure['layout']['sliders'] = {
    'args': [
        'transition', {
            'duration': 400,
            'easing': 'cubic-in-out'
        }
    ],
    'initialValue': region_days[0],
    'plotlycommand': 'animate',
    'values': region_days,
    'visible': True
}
figure['layout']['xaxis'] = {'range':[region_days[0], region_days[-1]]}
min_y = df_specific_region["Streams"].unique().tolist()
max_y = max(min_y)
min_y = min(min_y)
figure['layout']['yaxis'] = {'range':[min_y, max_y]}

figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 200, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 100, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Day:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 200, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

# make initial day data
date = region_days[0]
df_song_data_day = df_specific_region[df_specific_region["Date"] == date]
for i, row in df_song_data_day.iterrows():
    position_y = row["Streams"]
    position_x = date
    datadict = {
        'x' : [position_x],
        'y' : [position_y],
        'mode' :'markers',
        'name' : row["Track Name"]
    }
    figure['data'].append(datadict)

#making data for each frame
for date in region_days:
    frame = {'data': [], 'name': date}
    df_song_data_day = df_specific_region[df_specific_region["Date"] == date]
    for i, row in df_song_data_day.iterrows():
        position_y = row["Streams"]
        position_x = date
        data_dict = {
            'x' : [position_x],
            'y' : [position_y],
            'mode' :'markers',
            'name' : row["Track Name"]
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [date],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': date,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

iplot(figure, filename='anim')